In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [44]:
# Set up chrome
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Jon's PC\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


### Mars Article

In [24]:
# Set url and visit
url = 'https://redplanetscience.com'
browser.visit(url)
# Set optional delay for loading page
browser.is_element_present_by_css('div.list_text', wait_time = 1)

True

In [25]:
# Set up html parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [26]:
# Scrape for article title
slide_elem.find('div', class_= 'content_title')

<div class="content_title">NASA's Curiosity Keeps Rolling As Team Operates Rover From Home</div>

In [27]:
# Save text as new variable
news_title = slide_elem.find('div', class_= 'content_title').get_text()
news_title

"NASA's Curiosity Keeps Rolling As Team Operates Rover From Home"

In [28]:
# Retrieve article summary, as well
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'The team has learned to meet new challenges as they work remotely on the Mars mission.'

### Featured Images


In [29]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [30]:
# Find and click full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [31]:
# Parse html
html = browser.html
img_soup = soup(html, 'html.parser')

In [32]:
# Find relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [33]:
# Create url for image
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

### Mars Facts

In [34]:
# Create Dataframe from html from new site, first table
df = pd.read_html('https://galaxyfacts-mars.com')[0]
# Set columns
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [35]:
# Convert back to html code
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [36]:
# # Close browser
# browser.quit()

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [125]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

html = browser.html

In [126]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
hem_soup = soup(html, 'html.parser')
base_url= 'https://marshemispheres.com/'

In [127]:
# Scrape for images and titles, put into list of dictionaries
hemisphere_img = hem_soup.find_all('img', class_='thumb')
hemisphere_title = hem_soup.find_all('h3')
for x in range(0,4):
    dictionary={}
    end_url = hemisphere_img[0+x].get('src')
    dictionary["image"] = f'{base_url}{end_url}'
    dictionary["title"] = hemisphere_title[0+x].get_text()
    hemisphere_image_urls.append(dictionary)

In [128]:
# Check work
hemisphere_image_urls

[{'image': 'https://marshemispheres.com/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'image': 'https://marshemispheres.com/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'image': 'https://marshemispheres.com/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'image': 'https://marshemispheres.com/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [132]:
# Wrong images, we need jpgs
base_url = 'https://marshemispheres.com/'
# Empty List
hemisphere_image_urls = []
#Loop through the images on site
for x in range (3,7):
    # Create empty dictionary
    dictionary={}
    # Click image
    hemisphere_main = browser.find_by_tag('img')[x]
    hemisphere_main.click()
    # Parse new page
    html = browser.html
    hem_soup = soup(html, 'html.parser')
    # Find image url and title
    hemisphere_img = hem_soup.find_all('a', target="_blank")[-2].get('href')
    hemisphere_title = hem_soup.find('h2', class_="title").get_text()
    # Add to dictionary
    dictionary["image"] = f'{base_url}{hemisphere_img}'
    dictionary["title"] = hemisphere_title
    # Add dictionary to list
    hemisphere_image_urls.append(dictionary)
    # Go back to previous page
    browser.back()

In [134]:
# 4. Print the list that holds the dictionary of each image url and title.
print(hemisphere_image_urls)

[{'image': 'https://marshemispheres.com/images/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'image': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}, {'image': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}, {'image': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}]


In [135]:
# 5. Quit the browser
browser.quit()